![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important.

For this project, you're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

In [16]:
-- Write your query here
-- MA and moving sigma
WITH cte AS (
SELECT 
	item_no, 
	operator,
	height,
	ROW_NUMBER() OVER w AS row_number,
	AVG(height) OVER w AS avg_height,
STDDEV(height) OVER w AS stddev_height
FROM manufacturing_parts
WINDOW w AS (PARTITION BY operator ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)
	),
-- UCL and LCL	
cte2 AS (
SELECT 
	operator,
	row_number,
	height,
	ROUND(avg_height::numeric, 2) AS avg_height,
	ROUND(stddev_height::numeric, 2) AS stddev_height,
	ROUND((avg_height + 3*stddev_height/SQRT(5))::numeric,2) AS ucl,
	ROUND((avg_height - 3*stddev_height/SQRT(5))::numeric,2) AS lcl
FROM cte
-- filter out incomplete rows
WHERE row_number > 4
ORDER BY item_no
	)
SELECT
	operator,
	row_number,
	height,
	avg_height,
	stddev_height,
	ucl,
	lcl,
	CASE WHEN
		height BETWEEN lcl AND ucl THEN FALSE
		ELSE TRUE END
	AS alert
FROM cte2
	

,operator,row_number,height,avg_height,stddev_height,ucl,lcl,alert
0,Op-1,5,19.46,19.78,1.06,21.20,18.35,False
1,Op-1,6,20.36,19.91,1.09,21.38,18.45,False
2,Op-1,7,20.22,20.03,1.08,21.49,18.57,False
3,Op-1,8,21.03,19.93,0.93,21.18,18.68,False
4,Op-1,9,19.78,20.17,0.60,20.97,19.37,False
...,...,...,...,...,...,...,...,...
415,Op-20,9,20.51,20.36,0.86,21.52,19.21,False
416,Op-20,8,21.36,20.10,1.00,21.43,18.76,False
417,Op-20,7,19.30,20.15,1.08,21.59,18.70,False
418,Op-20,6,20.96,20.58,1.09,22.04,19.12,False
